# Optibook Manual

This notebook contains examples for **all interactions** you can do with optibook (that means inserting orders, getting your positions, etc.). You should use this notebook as a reference/documenation for the system later on when you write more extensive algorithms.

First we do some setup and import the optibook client, which is used to connect to the optibook exchange. If everything is setup correctly, you should see the line "Setup was successful." being printed.

In [16]:
from optibook.synchronous_client import Exchange

import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


### Define Your Instrument

An instrument is the term we use for the 'thing' that we are trading. As such, if we are trading BMW stocks, we would call those an instrument. However, BMW options would be a different instrument. 

This field determines which instrument we are trading. By changing it, you will insert trades for a different instrument. You can see all available instruments by looking at the dropdown menu labeled "Instruments" on the visualizer.

In [17]:
instrument_id = 'GOOGLE'

### Connect to Exchange

In [18]:
e = Exchange()
a = e.connect()

# you can also define host/user/pass yourself
# when not defined, it is taken from ~/.optibook file if it exists
# if that file does not exists, an error is thrown

#e = Exchange(host='host-to-connect-to')
#a = e.connect(username='your-username', password='your-password')


2021-03-06 21:15:29,253 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2021-03-06 21:15:29,396 [client    ] [Thread-4    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


## Outstanding Orders, Trades, Current Positions and PnL

In [19]:
# Returns all currently outstanding orders
orders = e.get_outstanding_orders(instrument_id)
for o in orders.values():
    print(o)

In [20]:
# Returns all trades you have done since the last time this function was called
trades = e.poll_new_trades(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [21]:
# Returns all trades you have done since since the instantiation of the Exchange
trades = e.get_trade_history(instrument_id)
for t in trades:
    print(f"[TRADED {t.instrument_id}] price({t.price}), volume({t.volume}), side({t.side})")

In [22]:
# Returns all current positions
positions = e.get_positions()
for p in positions:
    print(p, positions[p])

AMAZON 0
GOOGLE 0
TECH_BASKET 0


In [23]:
# Returns all current positions with cash invested
positions = e.get_positions_and_cash()
for p in positions:
    print(p, positions[p])

AMAZON {'volume': 0, 'cash': -12.600000000001387}
GOOGLE {'volume': 0, 'cash': -46.100000000002126}
TECH_BASKET {'volume': 0, 'cash': -256.1999999999989}


In [24]:
# Returns Current PnL based on last Traded Price
pnl = e.get_pnl()
print(pnl)

-314.9000000000024


## Order Book and Public Trade Ticks

In [33]:
import inspect
book = e.get_last_price_book(instrument_id)
print(len(book.asks))
print(len(book.bids))
for a in book.asks:
    print(a)
for b in book.bids:
    print(b)

print(inspect.getmembers(book))

1
1
[price_volume] price=98.4, volume=250
[price_volume] price=98.30000000000001, volume=250
[('__class__', <class 'optibook.common_types.PriceBook'>), ('__delattr__', <method-wrapper '__delattr__' of PriceBook object at 0x7f85bc110f28>), ('__dict__', {'timestamp': datetime.datetime(2021, 3, 6, 21, 18, 9, 883779), 'instrument_id': 'GOOGLE', 'bids': [[price_volume] price=98.30000000000001, volume=250], 'asks': [[price_volume] price=98.4, volume=250]}), ('__dir__', <built-in method __dir__ of PriceBook object at 0x7f85bc110f28>), ('__doc__', '\n    An order book at a specific point in time.\n\n    Attributes\n    ----------\n    timestamp: datetime.datetime\n        The time of the snapshot.\n\n    instrument_id: str\n        The id of the instrument the book is on.\n\n    bids: List[PriceVolume]\n        List of price points and volumes representing all bid orders.\n        Sorted from highest price to lowest price (i.e. from best to worst).\n\n    asks: List[PriceVolume]\n        List 

2021-03-06 21:18:32,497 [client    ] [Thread-5    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [28]:
# Returns all public tradeticks since the last time this function was called
tradeticks = e.poll_new_trade_ticks(instrument_id)
for t in tradeticks:
    print(f"[{t.instrument_id}] price({t.price}), volume({t.volume}), aggressor_side({t.aggressor_side}), buyer({t.buyer}), seller({t.seller})")

[GOOGLE] price(98.4), volume(5), aggressor_side(ask), buyer(), seller()
[GOOGLE] price(98.30000000000001), volume(1), aggressor_side(ask), buyer(), seller()
[GOOGLE] price(98.2), volume(1), aggressor_side(ask), buyer(), seller()
[GOOGLE] price(98.2), volume(5), aggressor_side(bid), buyer(), seller()
[GOOGLE] price(98.2), volume(245), aggressor_side(bid), buyer(), seller()
[GOOGLE] price(98.30000000000001), volume(50), aggressor_side(bid), buyer(), seller()
[GOOGLE] price(98.30000000000001), volume(1), aggressor_side(bid), buyer(), seller()
[GOOGLE] price(98.2), volume(5), aggressor_side(bid), buyer(), seller()
[GOOGLE] price(98.2), volume(5), aggressor_side(bid), buyer(), seller()
[GOOGLE] price(98.2), volume(5), aggressor_side(bid), buyer(), seller()
[GOOGLE] price(98.2), volume(5), aggressor_side(bid), buyer(), seller()
[GOOGLE] price(98.2), volume(5), aggressor_side(bid), buyer(), seller()
[GOOGLE] price(98.2), volume(5), aggressor_side(bid), buyer(), seller()
[GOOGLE] price(98.2), 

In [12]:
# Returns all public tradeticks since the instantiation of the Exchange
tradeticks = e.get_trade_tick_history(instrument_id)
for t in tradeticks:
    print(t)

In [13]:
# See all your outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    print(f"Outstanding order: order_id({o.order_id}), instrument_id({o.instrument_id}), price({o.price}), volume({o.volume}), side({o.side})")

## Inserting and Deleting Orders

In [14]:
# Insert bid limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=3000, volume=21, side='bid', order_type='limit')
print(f"Order Id: {result}")

Order Id: 312699


In [15]:
# Insert ask limit order - This trades against any current orders, and any remainders become new resting orders in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=3000, volume=1, side='ask', order_type='limit')
print(f"Order Id: {result}")

Order Id: 312708


In [16]:
# Insert bid IOC - This can trade against any resting volume but does not remain in the book
# Use this to buy.
result = e.insert_order(instrument_id, price=445.0, volume=1, side='bid', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 312720


In [17]:
# Insert ask IOC - This can trade against any resting volume but does not remain in the book
# Use this to sell.
result = e.insert_order(instrument_id, price=430.0, volume=1, side='ask', order_type='ioc')
print(f"Order Id: {result}")

Order Id: 312727


In [18]:
# Attempt to delete inserted order by order_id
order_id = 4
result = e.delete_order(instrument_id, order_id=order_id)
print()
print(f"Deleted order id {order_id}: {result}")


Deleted order id 4: False


In [19]:
# Change volume for existing order
order_id = 5
new_volume = 16
result = e.amend_order(instrument_id, order_id=order_id, volume=new_volume)
print(f"Changed volume for order id {order_id} to {new_volume} lots: {result}")

Changed volume for order id 5 to 16 lots: False


In [20]:
# Delete all outstanding orders
outstanding = e.get_outstanding_orders(instrument_id)
for o in outstanding.values():
    result = e.delete_order(instrument_id, order_id=o.order_id)
    print(f"Deleted order id {o.order_id}: {result}")

Deleted order id 312708: True


## 'Hack' Out of Positions

In [21]:
# Get out of all positions you are currently holding, regarless of the loss involved. That means selling whatever
# you are long, and buying-back whatever you are short. Be sure you know what you are doing when you use this logic.
print(e.get_positions())
for s, p in e.get_positions().items():
    if p > 0:
        e.insert_order(s, price=1, volume=p, side='ask', order_type='ioc')
    elif p < 0:
        e.insert_order(s, price=100000, volume=-p, side='bid', order_type='ioc')  
print(e.get_positions())

{'AMAZON': 0, 'GOOGLE': 22, 'TECH_BASKET': 0}
{'AMAZON': 0, 'GOOGLE': 0, 'TECH_BASKET': 0}
